In [0]:
import re
from pyspark.sql.types import *
import sys, os
from delta.tables import *
import glob


In [0]:
def Is_Numeric(value):        
    
    if str(value).isdigit() == True:
        return True
    else:
        return False 

spark.udf.register("IsNumeric", Is_Numeric, BooleanType())

In [0]:
def stringToBool(v):
  return v.lower() in ("yes", "true", "t", "1")

In [0]:
def UncacheTempTable(tempTableName):
  
  if spark._jsparkSession.catalog().tableExists('default', tempTableName) == True:
    spark.catalog.uncacheTable('tempTableName')

In [0]:
def Remove_Non_Alpha_Numeric(string_value):

  return re.sub(r'\W+', '', str(string_value))

spark.udf.register("RemoveNonAlphaNumeric", Remove_Non_Alpha_Numeric, StringType())

In [0]:
def OptimizeAndVacuumStandard(MountLocation):
  if DeltaTable.isDeltaTable(spark, MountLocation) == True:
    deltaTable = DeltaTable.forPath(spark, MountLocation)
    deltaTable.vacuum()
    spark.sql(f'OPTIMIZE delta.`{MountLocation}`')

In [0]:
def OptimizeAndVacuumZORDER(MountLocation, ColumnList):
  if DeltaTable.isDeltaTable(spark, MountLocation) == True:
    deltaTable = DeltaTable.forPath(spark, MountLocation)
    deltaTable.vacuum()
    spark.sql(f'OPTIMIZE delta.`{MountLocation}` ZORDER BY {ColumnList}')

In [0]:
def Remove_Non_Alpha(string_value):
  return ''.join(i for i in string_value if i.isalpha())

spark.udf.register("RemoveNonAlpha", Remove_Non_Alpha, StringType())

In [0]:
def NumberOfOutputRows(tablePath):
  
  lastOperationDF = DeltaTable.forPath(spark, tablePath).history(1)
  
  if lastOperationDF.select('operation').collect()[0][0] == "MERGE":
      rows = int(lastOperationDF.select('operationMetrics.numTargetRowsInserted').collect()[0][0]) + int(lastOperationDF.select('operationMetrics.numTargetRowsDeleted').collect()[0][0]) + int(lastOperationDF.select('operationMetrics.numTargetRowsUpdated').collect()[0][0]) 
  
  else:   
    rows = lastOperationDF.select('operationMetrics.numOutputRows').collect()[0][0]
  
  return str(rows)

In [0]:
def CheckFolderForDeltaTables(folderList_Input, NonDeltaIsExcluded):
  CheckFolderForDeltaTablesSchema = StructType([ StructField('folderName', StringType(),True),StructField('IsDeltaTable', BooleanType(),True) ])
  CheckFolderForDeltaTablesData = []
  CheckFolderForDeltaTables_df = spark.createDataFrame(data=CheckFolderForDeltaTablesData,schema=CheckFolderForDeltaTablesSchema)

  folderList = dbutils.fs.ls(folderList_Input)

  for folder in folderList:
    mountLocation = folder.path.replace("dbfs:", "")
    IsDeltaTable = DeltaTable.isDeltaTable(spark, mountLocation)
    data = [(mountLocation, IsDeltaTable)]

    newRow = spark.createDataFrame(data=data,schema=CheckFolderForDeltaTablesSchema)
    CheckFolderForDeltaTables_df = CheckFolderForDeltaTables_df.union(newRow)
    
  
  if NonDeltaIsExcluded == True:
    output = CheckFolderForDeltaTables_df.filter(CheckFolderForDeltaTables_df.IsDeltaTable == 'true')
  else:
    output = CheckFolderForDeltaTables_df
  
  return(output)

In [0]:
def listFilesInDirectory(path):
  results = []

  for path, subdirs, files in os.walk(path):
    for name in files:
        results.append([
                        path
                        ,name
                       ])
  return results

In [0]:
def listFilesInDirectoryWildcard(path):
  results = []

  for path in glob.glob(path):
      for path, subdirs, files in os.walk(path):
        for name in files:
            results.append([
                            path
                            ,name
                           ])
  return results